In [1]:
import pandas as pd
import numpy as np
import time
from influxdb import DataFrameClient
!pip install influxdb
!pip install requests
!pip install pycountry
!pip install countryinfo
import pycountry as pycountry
import requests
from pandas.io.json import json_normalize
import json
from datetime import datetime
from countryinfo import CountryInfo

In [2]:
class ApiData:
     def __init__(self,date,country,lat,lon,confirmed,death,recovered):
        self.date = date
        self.country = country
        self.lat = lat
        self.lon = lon
        self.confirmed = confirmed
        self.death = death
        self.recovered = recovered
        

In [3]:
def init_database():
    #get get v data
    for i in range(249):
        now=pd.to_datetime('now')
        name = list(pycountry.countries)[i].name
        name = name.replace(' ','')
        url = 'https://api.covid19api.com/live/country/{countryName}'.format(countryName =  name)
        dd = []
        df = pd.DataFrame(columns=['country','date','confirmed','death','recovered','lat','lon'])
        json_data = requests.get(url)
        if json_data.json() == []:
                print(name+' no data')
        else:
                print('okay')
                try:
                    dataLen = len(json_data.json())
                    for y in range(dataLen-1):
                        last = json_data.json()[y]
                        dd.append(ApiData(json_data.json()[y]['Date'],(name),last['Lat'],last['Lon'],last['Confirmed'],last['Deaths'],last['Recovered']))
                        population = CountryInfo(name).info()['population']
                        if y == 0:
                            df = pd.DataFrame(columns=['country','date','confirmed','death','recovered','lat','lon'])
                        df.loc[y] = [dd[y].country,dd[y].date,dd[y].confirmed,dd[y].death,dd[y].recovered,dd[y].lat,dd[y].lon]
                    df = pd.DataFrame(df).set_index('date')
                    df.index = pd.to_datetime(df.index)
                    df['confirmed']=df['confirmed'].astype(int)
                    df['death']=df['death'].astype(int)
                    df['recovered']=df['recovered'].astype(int)
                    df['lat']=df['lat'].astype(float)
                    df['lon']=df['lon'].astype(float)

                    print (df)
                    client.write_points(df, 'database', protocol=protocol,
                                                field_columns=['confirmed','death','recovered','lat','lon'],
                                                tag_columns=['country']) 

                except:
                    print('crash')

In [4]:
init_database()

Aruba no data
okay
                               country  confirmed  death  recovered    lat  \
date                                                                         
2020-04-13 00:00:00+00:00  Afghanistan        607     18         32  33.94   
2020-04-14 00:00:00+00:00  Afghanistan        665     21         32  33.94   
2020-04-15 00:00:00+00:00  Afghanistan        714     23         40  33.94   
2020-04-16 00:00:00+00:00  Afghanistan        840     30         54  33.94   
2020-04-17 00:00:00+00:00  Afghanistan        840     30         54  33.94   
2020-04-18 00:00:00+00:00  Afghanistan        906     30         99  33.94   
2020-04-19 00:00:00+00:00  Afghanistan        933     30        112  33.94   
2020-04-20 00:00:00+00:00  Afghanistan        996     33        131  33.94   
2020-04-21 00:00:00+00:00  Afghanistan       1026     36        135  33.94   
2020-04-22 00:00:00+00:00  Afghanistan       1092     36        150  33.94   
2020-04-23 00:00:00+00:00  Afghanistan       

In [2]:

def init_Database_Pop():
    population = []
    for i in range(140):
        try:
            population.append(CountryInfo(data[i].country).info()['population'])
        except:
            population.append(0)


    #get data
    data = []
    for i in range(249):
        now=pd.to_datetime('now')
        name = list(pycountry.countries)[i].name
        name = name.replace(' ','')
        url = 'https://api.covid19api.com/live/country/{countryName}'.format(countryName =  name)
        json_data = requests.get(url)
        if json_data.json() == []:
            print(name+' no data')
        else:
            print('okay')
            for y in range(len(json_data.json())):
                last = json_data.json()[y]  
                data.append(ApiData(now,(list(pycountry.countries)[i].name),last['Lat'],last['Lon'],last['Confirmed'],last['Deaths'],last['Recovered']))
                population = []
                try:
                    population.append(CountryInfo(data[y].country).info()['population'])
                except:
                    population.append(0)

                df = pd.DataFrame(columns=['country','date','confirmed','death','recovered','lat','lon','population'])

                df.loc[y] = [data[y].country,data[y].date,data[y].confirmed,data[y].death,data[y].recovered,data[y].lat,data[y].lon,population[y]]
                df = df[df.population != 0]


                df = pd.DataFrame(df).set_index('date')
                df.index = pd.to_datetime(df.index)
                df['confirmed']=df['confirmed'].astype(int)
                df['death']=df['death'].astype(int)
                df['recovered']=df['recovered'].astype(int)
                df['lat']=df['lat'].astype(float)
                df['lon']=df['lon'].astype(float)


                #get data every day
                p = df[['confirmed','death','recovered']].div(df.population, axis=0)
                p['confirmed'] = p['confirmed'].astype(float)
                p['death'] = p['death'].astype(float)
                p['recovered'] = p['recovered'].astype(float)
                p = p.round(6)
                p['country'] = df['country'] 
                print(df)
                p['Date'] = df['Date']
                dataPpopulation = p
                dataPpopulation = pd.DataFrame(dataPpopulation).set_index('date')
                dataPpopulation['death'] = dataPpopulation['death']*1000000000
                dataPpopulation['confirmed'] = dataPpopulation['confirmed']*1000000000
                dataPpopulation['recovered'] = dataPpopulation['recovered']*1000000000
                dataPpopulation

                #get data every day
                user = 'root'
                password = 'root'
                host='influxdb' #172.28.1.2 #localhost #3.14.5.146 - kozos osztaly InfluxDB szerver
                port=8086
                dbname='covidWorP' #az en adatbazis nevem
                protocol = 'line' #json
                measurement='teszt'
                client = DataFrameClient(host, port, user, password, dbname)
                client.drop_database(dbname)
                client.drop_retention_policy(dbname)
                client.create_database(dbname)
                client.create_retention_policy(dbname, '1000d', 1, default=True)
                client.query('show databases')

                client.write_points(dataPpopulation, 'database', protocol=protocol,
                                        field_columns=['confirmed','death','recovered',],
                                        tag_columns=['country'])

    

In [ ]:
init_Database_Pop()